In [1]:
from google.colab import drive
drive.mount('/content/drive')
import zipfile, os, urllib.request, glob, math, shutil, sys
import torchvision
import torch.optim as optim
from torchvision import datasets, models, transforms
from sklearn.metrics import confusion_matrix, balanced_accuracy_score, precision_recall_fscore_support, roc_auc_score, accuracy_score
import sklearn.metrics
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
import matplotlib.pyplot as plt
import PIL
import PIL.Image

sys.path.append('drive/MyDrive/TrainModels')
import modelArchs

fold_number = 1
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

Mounted at /content/drive


In [ ]:
train_dataset = pd.read_csv('drive/MyDrive/train_dataset_app_Fold_' + str(fold_number) + '.csv')
test_dataset = pd.read_csv('drive/MyDrive/test_dataset_app_Fold_' + str(fold_number) + '.csv')
print(len(train_dataset))
print(len(test_dataset))

1833
465




Οι εικόνες είναι αποθηκευμένες στο drive σε zip.

Υπάρχουν 16 zip files, ένα για κάθε κλάση από το train και test dataset.

Κάνω extract τις εικόνες.


In [ ]:
os.makedirs('train_imgs')
os.makedirs('test_imgs')

prefix = 'drive/MyDrive/'

for i in ['train_imgs', 'test_imgs']:
  for j in ['ACK', 'BCC', 'MEL', 'NEV', 'SCC', 'SEK']:

    name = i + '_' + j
    src_zip = prefix + name + '.zip'
    dest_zip = i + '/' + j

    with zipfile.ZipFile(src_zip, 'r') as zip_ref:
      zip_ref.extractall(dest_zip)

    print("Finished with: ", name)

Finished with:  train_imgs_ACK
Finished with:  train_imgs_BCC
Finished with:  train_imgs_MEL
Finished with:  train_imgs_NEV
Finished with:  train_imgs_SCC
Finished with:  train_imgs_SEK
Finished with:  test_imgs_ACK
Finished with:  test_imgs_BCC
Finished with:  test_imgs_MEL
Finished with:  test_imgs_NEV
Finished with:  test_imgs_SCC
Finished with:  test_imgs_SEK


Κάνω load το εκάστοτε pretrained μοντέλο (MobileNet_V3_Large , Efficient Net,Resnet34, GoogleNet, MobileNet_V3_Large + SA(https://pytorch.org/vision/stable/models.html#classification), trained πάνω στο imagenet. Στο τελευταίο layer αλλάζουμε τα out_features από 1000 σε 6, καθώς έχουμε 6 διαφορετικές κλάσεις προς ταξινόμηση.

In [2]:
model = modelArchs.get_MobileNetSa(6)
model.to(device)

Downloading: "https://download.pytorch.org/models/mobilenet_v3_large-5c1a4163.pth" to /root/.cache/torch/hub/checkpoints/mobilenet_v3_large-5c1a4163.pth
100%|██████████| 21.1M/21.1M [00:00<00:00, 76.2MB/s]


MobileNetV3(
  (features): Sequential(
    (0): Conv2dNormActivation(
      (0): Conv2d(3, 16, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
      (1): BatchNorm2d(16, eps=0.001, momentum=0.01, affine=True, track_running_stats=True)
      (2): Hardswish()
    )
    (1): InvertedResidual(
      (block): Sequential(
        (0): Conv2dNormActivation(
          (0): Conv2d(16, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), groups=16, bias=False)
          (1): BatchNorm2d(16, eps=0.001, momentum=0.01, affine=True, track_running_stats=True)
          (2): ReLU(inplace=True)
        )
        (1): Conv2dNormActivation(
          (0): Conv2d(16, 16, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (1): BatchNorm2d(16, eps=0.001, momentum=0.01, affine=True, track_running_stats=True)
        )
      )
    )
    (2): InvertedResidual(
      (block): Sequential(
        (0): Conv2dNormActivation(
          (0): Conv2d(16, 64, kernel_size=(1, 1), stride=(1, 1), bi

Ορίζω διάφορα transformations για να κάνουμε augment το training set, ώστε να πετύχουμε καλύτερο generalization του μοντέλου.

In [ ]:
transforms_train = transforms.Compose([
    transforms.RandomResizedCrop(size = 224, scale = (0.4,1.0)),
    transforms.RandomHorizontalFlip(),
    transforms.RandomVerticalFlip(),
    transforms.ColorJitter(brightness = [0.7, 1.3], contrast = [0.7, 1.3], saturation = [0.7, 1.3], hue = [-0.1,0.1]),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])

transforms_test = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])

Ορισμός υπερ-παραμέτρων.

In [ ]:
epochs = 30
batch_len = 32

criterion = nn.CrossEntropyLoss()
learning_rate = 0.001
optimizer = optim.Adam(model.parameters(), lr=learning_rate)
scheduler = optim.lr_scheduler.StepLR(optimizer, step_size = 5, gamma=0.2)

Εδώ ορίζω τα dataset και dataloaders, χρησιμοποιώντας το weightRandomSampler για να δημιουργήσει balanced batches:

https://towardsdatascience.com/demystifying-pytorchs-weightedrandomsampler-by-example-a68aceccb452

In [ ]:
def get_weights_for_imgs(images, n_categories):

    n_images = len(images)

    imgs_per_category = [0] * n_categories
    for _, image_category in images:
      imgs_per_category[image_category] += 1

    weight_per_category = [0] * n_categories # έχουμε 6 κατηγορίες
    for i in range(n_categories):
      weight_per_category[i] = 1 / float(imgs_per_category[i])#float(n_images) / float(imgs_per_category[i])

    weights = [0] * n_images
    for idx, (_, image_category) in enumerate(images):
        weights[idx] = weight_per_category[image_category]

    return weights

train_dataset = datasets.ImageFolder('train_imgs', transforms_train)
test_dataset = datasets.ImageFolder('test_imgs', transforms_test)

len_train_dataset = len(train_dataset)
len_test_dataset = len(test_dataset)

print("Length of train_dataset: ", len_train_dataset)
print("Length of test dataset: ", len_test_dataset)

weights = get_weights_for_imgs(train_dataset.imgs, len(train_dataset.classes))
weights = torch.DoubleTensor(weights)
train_sampler = torch.utils.data.sampler.WeightedRandomSampler(weights, len(weights))

train_dataloader = torch.utils.data.DataLoader(train_dataset, batch_size = batch_len, sampler = train_sampler, pin_memory=True)
test_dataloader = torch.utils.data.DataLoader(test_dataset, batch_size = batch_len, shuffle=False)

Length of train_dataset:  1833
Length of test dataset:  465


['ACK', 'BCC', 'MEL', 'NEV', 'SCC', 'SEK']

Ορίζω συναρτήσεις για υπολογισμό του accuracy και του confusion matrix πάνω στο test set. Στη συνέχεια θα δημιουργήσω και άλλες συναρτήσεις για υπολογισμό F1-score, sensitivity, specificity, precision.

In [ ]:
def test_model(model,test_dataloader):

  pred_list = []
  label_list = []
  output_probs = []
  metrics = []

  for i, (inputs, labels) in enumerate(test_dataloader,1):

    label_list.extend(labels.data.tolist())

    inputs = inputs.to(device)
    labels = labels.to(device)
    outputs = model(inputs)
    probs = torch.softmax(outputs.data.cpu(), dim = 1).tolist()
    output_probs.extend(probs)

    pred =  torch.max(outputs, 1)[1].data.cpu().tolist()
    pred_list.extend(pred)

  conf_matrix = confusion_matrix(label_list , pred_list)

  metrics.append(accuracy_score(label_list, pred_list))
  metrics.extend(precision_recall_fscore_support(label_list,pred_list, average = 'macro'))
  metrics.append(roc_auc_score(label_list,output_probs, average = 'macro', multi_class = 'ovr'))
  metrics.extend(precision_recall_fscore_support(label_list,pred_list, average = 'weighted'))
  metrics.append(roc_auc_score(label_list,output_probs, average = 'weighted', multi_class = 'ovr'))

  return metrics, conf_matrix

Το βασικό train loop.

split se lesion id, mobile net + sa

In [ ]:
for epoch in range(1, epochs + 1):

  model.train()
  train_running_loss = 0

  #Training phase
  print("Starting epoch: " , epoch)

  for i, (inputs, labels) in enumerate(train_dataloader,1):
      inputs = inputs.to(device)
      labels = labels.to(device)

      optimizer.zero_grad()
      outputs = model(inputs)
      loss = criterion(outputs, labels)
      loss.backward()
      optimizer.step()

      train_running_loss += loss.detach().item()

  print('Epoch: %d | Loss: %.4f ' %(epoch, train_running_loss / i))
  scheduler.step()

  model.eval()
  try:
    metrics,conf_matrix = test_model(model,test_dataloader)
    print('Test metrics on epoch %d:' %(epoch))
    print(metrics)
    print("Confusion matrix: ")
    print(conf_matrix)
  except:
    print("Error in calculating metrics")

  try:
    checkpoint = {'model': model.state_dict(), 'epoch' : epoch}
    checkpoint_fn = 'drive/MyDrive/checkpoint' + str(epoch) + '.pt'
    torch.save(checkpoint, checkpoint_fn)
  except:
    print("Error in writing general checkpoint")

Starting epoch:  1
Epoch: 1 | Loss: 1.1463 
Test metrics on epoch 1:
[0.44731182795698926, 0.3726311878499678, 0.42030948265008666, 0.3108193805092186, None, 0.7711806233015327, 0.5432154155408603, 0.44731182795698926, 0.43885673375615286, None, 0.7494095964134165]
Confusion matrix: 
[[ 49  62  24  13   0   1]
 [  8 111  26  27   1   2]
 [  0   0   7   4   0   0]
 [  0   1   6  36   0   1]
 [  3  26   5   2   0   2]
 [  2   2  22  17   0   5]]
Starting epoch:  2
Epoch: 2 | Loss: 0.8839 
Test metrics on epoch 2:
[0.5161290322580645, 0.43585164835164836, 0.4869947475206218, 0.4219194946908926, None, 0.8148488479674582, 0.5923506242861081, 0.5161290322580645, 0.5263641641203066, None, 0.8127497579116595]
Confusion matrix: 
[[ 67  57   1   1  13  10]
 [  7 111  10  12  22  13]
 [  0   0   7   1   0   3]
 [  1   2  13  15   0  13]
 [  1  24   4   3   5   1]
 [  2   1   7   3   0  35]]
Starting epoch:  3
Epoch: 3 | Loss: 0.7358 
Test metrics on epoch 3:
[0.41935483870967744, 0.46965756083559

KeyboardInterrupt: ignored

split se patient id me mobilenet + sa

In [ ]:
for epoch in range(1, epochs + 1):

  model.train()
  train_running_loss = 0

  #Training phase
  print("Starting epoch: " , epoch)

  for i, (inputs, labels) in enumerate(train_dataloader,1):
      inputs = inputs.to(device)
      labels = labels.to(device)

      optimizer.zero_grad()
      outputs = model(inputs)
      loss = criterion(outputs, labels)
      loss.backward()
      optimizer.step()

      train_running_loss += loss.detach().item()

  print('Epoch: %d | Loss: %.4f ' %(epoch, train_running_loss / i))
  scheduler.step()

  model.eval()
  try:
    metrics,conf_matrix = test_model(model,test_dataloader)
    print('Test metrics on epoch %d:' %(epoch))
    print(metrics)
    print("Confusion matrix: ")
    print(conf_matrix)
  except:
    print("Error in calculating metrics")

  #try:
  #  checkpoint = {'model': model.state_dict(), 'epoch' : epoch}
  #  checkpoint_fn = 'drive/MyDrive/checkpoint' + str(epoch) + '.pt'
  #  torch.save(checkpoint, checkpoint_fn)
  #except:
  #  print("Error in writing general checkpoint")

Starting epoch:  1
Epoch: 1 | Loss: 1.2854 
Test metrics on epoch 1:
[0.3774403470715835, 0.4962009858002694, 0.5067140698087145, 0.3921639606022172, None, 0.8222551032265266, 0.614688595754037, 0.3774403470715835, 0.4125040229502053, None, 0.7826674464124928]
Confusion matrix: 
[[ 64  27   0   0  46  10]
 [  6  40   2   0 122   7]
 [  0   0  10   1   0   1]
 [  1   2  17  12   6  13]
 [  3   2   0   0  25   4]
 [  3   2   7   2   3  23]]
Starting epoch:  2
Epoch: 2 | Loss: 1.0511 
Test metrics on epoch 2:
[0.5661605206073753, 0.5167056702078253, 0.6232795660637136, 0.5437041951678462, None, 0.8688057322451154, 0.6143531284243942, 0.5661605206073753, 0.5747704429474465, None, 0.8485277847652226]
Confusion matrix: 
[[91 21  5  8 11 11]
 [42 82  5  5 38  5]
 [ 0  1 10  1  0  0]
 [ 0  5  5 34  1  6]
 [10  6  0  4 13  1]
 [ 2  1  2  3  1 31]]
Starting epoch:  3
Epoch: 3 | Loss: 0.8962 
Test metrics on epoch 3:
[0.6268980477223427, 0.5350313903069809, 0.5882906835050404, 0.5335450982160386,

KeyboardInterrupt: ignored

In [ ]:
!nvidia-smi

Wed Sep 27 22:28:36 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.105.17   Driver Version: 525.105.17   CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   51C    P0    27W /  70W |   5027MiB / 15360MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

Ένα μικρό τέστ για να δώ ότι ο dataloader παράγει σωστά τα shapes των tensors, και ότι κάθε batch είναι ισορροπημένο. Το τεστ έγινε πριν το train.

In [ ]:
from collections import Counter

for inputs, labels in train_dataloader:
  inputs = inputs.to(device)
  print("Input size:", inputs.shape)
  out = model(inputs)
  print("Output size: ", out.shape)

  labels = labels.tolist()
  print("Test if batch is balanced: ")
  print(Counter(labels).keys())
  print(Counter(labels).values())
  break